In [ ]:
import random

from src.data_generator.csv_data_processor import CSVDataProcessor
from src.utils.statistcs.statistical_functions import create_distributed_age_df
from spark_instance import spark
from src.utils.statistcs.data_visualisations import plot_age_distribution_with_sd, plot_kernel_density_age_distribution
from pyspark.sql.types import StructType, StructField, StringType, FloatType

from pyspark.sql.functions import col
  

In [ ]:
average_age = 40.2

csv_age_file_file = CSVDataProcessor(spark, "data/uk_age_population.csv")

csv_age_uk_df = csv_age_file_file.runner()

In [ ]:
csv_age_uk_df.show()

In [ ]:
plot_age_distribution_with_sd(csv_age_uk_df, average_age)


In [ ]:
plot_kernel_density_age_distribution(csv_age_uk_df)


In [ ]:
sampled_df = create_distributed_age_df(spark,  "data/uk_age_population.csv", 5000)

In [ ]:
sampled_df_aggregated = sampled_df.groupby(col("Age")).count()

In [ ]:
sampled_df_aggregated.orderBy("Age").show(n=200)

In [ ]:
plot_age_distribution_with_sd(sampled_df_aggregated)

In [ ]:
sampled_df.createOrReplaceGlobalTempView("sampled_df_with_unique_id_gt")

In [ ]:
from src.constants.admission_types_dataset import admission_mapping
flattened = [
    (
        top_level.name,
        sub_level_key.name,
        list(sub_level_info.get("stay_types")),
        sub_level_info.get("tests"),  
        random.choice(sub_level_info.get("doctors"))
    )
    for top_level, sub_level_dict in admission_mapping.items()
    for sub_level_key, sub_level_info in sub_level_dict.items()
]



In [ ]:
flattened


In [ ]:
from src.constants.condition_probabilities import condition_age_probability_dict

flattened_condition_probabilities = [
    (
        sub_admission,
        condition,
        gender if gender in ['male', 'female'] else None,
        float(age_range[0]),
        float(age_range[1]) if len(age_range) > 1 else float('inf'),
        float(probability) 
    )
    for sub_admission, conditions in condition_age_probability_dict.items()
    for condition, genders_or_age_prob_list in conditions.items()
    for gender, age_prob_list in (genders_or_age_prob_list.items() if isinstance(genders_or_age_prob_list, dict) else [(None, genders_or_age_prob_list)])
    for age_range, probability in age_prob_list
]



In [ ]:
flattened_condition_probabilities

In [ ]:
columns = ["condition_admission_type", "condition", "gender", "age_min", "age_max", "probability"]
schema = StructType([
    StructField("condition_admission_type", StringType(), True),
    StructField("condition", StringType(), True),
    StructField("condition_gender", StringType(), True),  # Assuming gender can be specific probability for gender 'male', 'female', or null
    StructField("age_min", FloatType(), False),
    StructField("age_max", FloatType(), False),
    StructField("probability", FloatType(), False)
])

condition_prob_df = spark.createDataFrame(data=flattened_condition_probabilities, schema=schema)


In [ ]:
condition_prob_df.show()

In [ ]:
"""
STEP 1 
    From the flattened data list we wish to create a DataFrame. This contains all the possible combinations for the given 
    top level admissions, sub level admissions, stay types and list of tests available from the admission_mapping, stay_type and admission_tests lists or dictionaries in admission_types_test_dataset.py
"""

mapping_df = spark.createDataFrame(flattened, ["top_level_admission", "sub_level_admission", "stay_types", "possible_tests", "doctor"])

joined_tbl = mapping_df.join(condition_prob_df, on=[mapping_df.sub_level_admission == condition_prob_df.condition_admission_type], how="left")

In [ ]:
gt_df = spark.sql("SELECT * FROM global_temp.sampled_df_with_unique_id_gt")


In [ ]:
"""
STEP 2
Create Enum class df and join it on to main driver df. 
Then create conditions 
"""

from src.data_generator.conditions_creator import ConditionsCreator
from src.utils.thread_operations import runner
from src.constants.type_constants import DepartmentTypes

enum_values = [e.name for e in DepartmentTypes]

enum_df = spark.createDataFrame(enum_values, StringType()).toDF("admission_type")

selected_conditions_df = runner(spark, ConditionsCreator, gt_df, joined_tbl, enum_df)


In [ ]:
from utils.util_funcs import get_row_count

get_row_count(selected_conditions_df)

In [ ]:
selected_conditions_df.show()

In [ ]:
# TODO:  
# tests to be chosen
# admission date to be chosen 
# hospital name 
# patient postcode 
# ethnicity distribution  
# _________________
# Long term 
# investigation which disease affects specific ethnicities
# blood type udf would need to consider ethnicity also eventually



In [ ]:
spark.stop()